# Yellow Fever Epidemic Modeling: Senegal 2002 Outbreak 

This notebook demonstrates the application of a compartmental SEIR  model with vaccination intervention of the Yellow Fever outbreak in Touba, Senegal (population: 800,000). The data was sourced from WHO archives. 

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from pathlib import Path 
import json 
import warnings 
warnings.filterwarnings('ignore')

import sys, os 
sys.path.append(os.path.abspath(os.path.join('..','src')))
from epimodels.yellow_fever_models import YellowFeverModel, create_vaccination_function
from epimodels.utils.parameter_estimation import ParameterEstimator, calculate_model_metrics

# set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Data Loading and Preparation

In [3]:
# Historical WHO outbreak data

data = {
    'date': ['2002-01-18', '2002-10-04', '2002-10-11', '2002-10-17', '2002-10-24', '2002-10-31', '2002-11-20', '2002-11-28'],
    'days_since_start': [0, 259, 266, 272, 279, 286, 306, 314],
    'cases_cumulative': [18, 12, 15, 18, 41, 45, 57, 60],
    'deaths_cumulative': [0, 0, 2, 2, 4, 4, 10, 11],
    'new_cases': [18, 12, 3, 3, 23, 4, 12, 3],
    'new_deaths': [0, 0, 2, 0, 2, 0, 6, 1]
}

data = pd.DataFrame(data)
data['date'] = pd.to_datetime(data['date'])

print("Historical Data Overview:")
print(f"Number of observations: {len(data)}")
print(f"Time span: {data['days_since_start'].min()} to {data['days_since_start'].max()} days")
print(f"Date range: {data['date'].min().date()} to {data['date'].max().date()}")
print(f"Total cases: {data['cases_cumulative'].max()}")
print(f"Total deaths: {data['deaths_cumulative'].max()}")
print(f"Case Fatality Rate: {data['deaths_cumulative'].iloc[-1] / data['cases_cumulative'].iloc[-1] * 100:.1f}%")

print("\nComplete Dataset:")
display(data)

Historical Data Overview:
Number of observations: 8
Time span: 0 to 314 days
Date range: 2002-01-18 to 2002-11-28
Total cases: 60
Total deaths: 11
Case Fatality Rate: 18.3%

Complete Dataset:


,date,days_since_start,cases_cumulative,deaths_cumulative,new_cases,new_deaths
0,2002-01-18,0,18,0,18,0
1,2002-10-04,259,12,0,12,0
2,2002-10-11,266,15,2,3,2
3,2002-10-17,272,18,2,3,0
4,2002-10-24,279,41,4,23,2
5,2002-10-31,286,45,4,4,0
6,2002-11-20,306,57,10,12,6
7,2002-11-28,314,60,11,3,1


### Summary Statistics

In [4]:
summary_stats = {
    'Total Cases': data['cases_cumulative'].max(),
    'Total Deaths': data['deaths_cumulative'].max(),
    'Case Fatality Rate (%)': round(data['deaths_cumulative'].iloc[-1] / data['cases_cumulative'].iloc[-1] * 100, 2),
    'Peak Daily Cases': data['new_cases'].max(),
    'Peak Daily Deaths': data['new_deaths'].max(),
    'Observation Period (days)': data['days_since_start'].max(),
    'Attack Rate (%)': round(data['cases_cumulative'].max() / 800000 * 100, 4)
}

summary_df = pd.DataFrame(list(summary_stats.items()), columns=['Metric', 'Value'])
display(summary_df)

,Metric,Value
0,Total Cases,60.0000
1,Total Deaths,11.0000
2,Case Fatality Rate (%),18.3300
3,Peak Daily Cases,23.0000
4,Peak Daily Deaths,6.0000
5,Observation Period (days),314.0000
6,Attack Rate (%),0.0075


## Model Setup

### Model Structure

For this analysis, I use a compartmental SEIR model with vaccination: 

**Compartments**
- **S**: Susceptible individuals
- **E**: Exposed individuals (infected, but not yet infectious)
- **I**: Infectious individuals 
- **R**: Recovered individuals (natural immunity)
- **V**: Vaccinated (vaccine-induced immunity)
- **D**: Deaths (cumulative) 

**Parameters**
- $\beta$: Transmission rate 
- $\sigma$: Incubation period (days) 
- $\gamma$: Infectious period (days) 
- $\alpha$: Mortality rate 
### Differential Equations: 

$$\frac{dS}{dt} = -\beta \frac{SI}{N} - V(t)S$$
$$\frac{dE}{dt} = \beta \frac{SI}{N} - \sigma E$$
$$\frac{dI}{dt} = \sigma E - \gamma I - \alpha I$$
$$\frac{dR}{dt} = \gamma I $$
$$\frac{dV}{dt} = V(t)S $$
$$\frac{dD}{dt} = \alpha I $$





In [5]:
# vaccination function starting on October 1, 2002 (day 259)
vaccination_func = create_vaccination_function(
    start_date=259,
    vaccination_rate=0.01,       # 1% of susceptibles per day
    ramp_duration=7.0,
    vaccine_efficiency=0.95
)

# initialize model with example parameters
# Initialize model with example parameters
model_example = YellowFeverModel(
    beta=0.5,           # Transmission rate
    sigma=1/6,          # Incubation period ~6 days
    gamma=1/7,          # Infectious period ~7 days
    alpha=0.15/7,       # Mortality rate (CFR ~15%)
    vaccination_func=vaccination_func,
    population=800000
)

print(model_example)
print(f"\nWith these parameters, R₀ = {model_example.calculate_r0():.4f}")


YellowFeverModel(
  β=0.5000 (transmission rate)
  σ=0.1667 (1/incubation)
  γ=0.1429 (recovery rate)
  α=0.0214 (mortality rate)
  N=800000 (population)
  R₀=3.0435
)

With these parameters, R₀ = 3.0435


## Parameter Estimation via MLE

Fit the model to the actual Senegalese outbreak data using Maximum Likelihood Estimation (MLE). 

### Setup Parameter Estimator

In [8]:
# initialize parameter estimator
estimator = ParameterEstimator(
    data=data,
    population=800000,
    fixed_params={
        'sigma':1/6,    # biorealistic six day incubation period
        'gamma':1/7,    # biorealistic seven day infectious period 
    }
)

print("Parameter Estimator initialized")
print(f"Fixed parameters: {estimator.fixed_params}")
print(f"Data points: {len(estimator.t_data)}")

Parameter Estimator initialized
Fixed parameters: {'sigma': 0.16666666666666666, 'gamma': 0.14285714285714285}
Data points: 8


### Define Initial Parameter Guess

In [9]:
# initial parameter guess
initial_guess = {
    'beta': 0.5,                # transmission rate
    'alpha': 0.02,              # mortality rate (CFR ~18%)
    'vaccination_rate': 0.01,    # 1% vaccinated per day
    'vaccination_start': 259     # Oct 1, 2002
}

print("Initial parameter guess:")
for param, value in initial_guess.items():
    print(f"  {param:20s}: {value:.6f}")

# test simulation with initial guess
cases_init, deaths_init = estimator.simulate_model(initial_guess, initial_infected=10)

print("\nInitial guess predictions:")
print(f"  Final cases:  {cases_init[-1]:.0f}  (Observed: {data['cases_cumulative'].iloc[-1]})")
print(f"  Final deaths: {deaths_init[-1]:.0f}  (Observed: {data['deaths_cumulative'].iloc[-1]})")

Initial parameter guess:
  beta                : 0.500000
  alpha               : 0.020000
  vaccination_rate    : 0.010000
  vaccination_start   : 259.000000

Initial guess predictions:
  Final cases:  756048  (Observed: 60)
  Final deaths: 92848  (Observed: 11)


### Run Parameter Optimization (MLE)

In [10]:
print("Running Maximum Likelihood Estimation...")

# fit model to data
fitted_params, result = estimator.fit_mle(
    initial_guess=initial_guess,
    method='L-BFGS-B',
    use_global=False
)

# display results
print("PARAMETER ESTIMATION RESULTS")
print("\nFitted Parameters:")
for name, value in fitted_params.items():
    print(f"  {name:20s}: {value:.6f}")

print("\nFixed Parameters:")
for name, value in estimator.fixed_params.items():
    print(f"  {name:20s}: {value:.6f}")

print("\nOptimization Info:")
for key, value in result.items():
    if value is not None:
        print(f"  {key:25s}: {value}")

# calculate R0 with fitted parameters
all_params = {**estimator.fixed_params, **fitted_params}
vaccination_func_fitted = create_vaccination_function(
    start_date=fitted_params['vaccination_start'],
    vaccination_rate=fitted_params['vaccination_rate']
)

model_fitted = YellowFeverModel(
    beta=fitted_params['beta'],
    sigma=all_params['sigma'],
    gamma=all_params['gamma'],
    alpha=fitted_params['alpha'],
    vaccination_func=vaccination_func_fitted,
    population=800000
)

r0 = model_fitted.calculate_r0()
print(f"Basic Reproduction Number R₀ = {r0:.4f}")

Running Maximum Likelihood Estimation...
PARAMETER ESTIMATION RESULTS

Fitted Parameters:
  beta                : 0.102606
  alpha               : 0.000000
  vaccination_rate    : 0.049736
  vaccination_start   : 259.000000

Fixed Parameters:
  sigma               : 0.166667
  gamma               : 0.142857

Optimization Info:
  success                  : True
  neg_log_likelihood       : 177.783434989267
  AIC                      : 363.566869978534
  BIC                      : 366.6572248674931
  n_iterations             : 40
  message                  : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Basic Reproduction Number R₀ = 0.7182


### Simulate Fitted Model

In [11]:
# simulate with fitted parameters
initial_conditions = {
    'S': 799990,
    'E': 0,
    'I': 10,
    'R': 0,
    'V': 0,
    'D': 0
}

t_eval = np.linspace(0, 350, 351)
t_fitted, y_fitted = model_fitted.simulate(
    initial_conditions=initial_conditions,
    t_span=(0, 350),
    t_eval=t_eval
)

print("Fitted model simulation complete")
print(f"  Time points: {len(t_fitted)}")
print(f"  Simulation span: {t_fitted[0]:.0f} to {t_fitted[-1]:.0f} days")

Fitted model simulation complete
  Time points: 351
  Simulation span: 0 to 350 days


### Evaluate Model Fit Quality

In [12]:
# calculate fit metrics
from scipy.interpolate import interp1d

S_fit, E_fit, I_fit, R_fit, V_fit, D_fit = y_fitted
cases_model = E_fit + I_fit + R_fit + D_fit

# interpolate to data time points
f_cases = interp1d(t_fitted, cases_model, kind='cubic')
f_deaths = interp1d(t_fitted, D_fit, kind='cubic')

t_data = data['days_since_start'].values
cases_pred = f_cases(t_data)
deaths_pred = f_deaths(t_data)

# calculate metrics
metrics_cases = calculate_model_metrics(data['cases_cumulative'].values, cases_pred)
metrics_deaths = calculate_model_metrics(data['deaths_cumulative'].values, deaths_pred)

print("Model Fit Quality:")
print("\nCases:")
for metric, value in metrics_cases.items():
    print(f"  {metric:15s}: {value:.4f}")

print("\nDeaths:")
for metric, value in metrics_deaths.items():
    print(f"  {metric:15s}: {value:.4f}")

# Create comparison table
comparison = pd.DataFrame({
    'Date': data['date'],
    'Observed Cases': data['cases_cumulative'],
    'Predicted Cases': cases_pred.round(1),
    'Observed Deaths': data['deaths_cumulative'],
    'Predicted Deaths': deaths_pred.round(1)
})

print("Observed vs Predicted:")
display(comparison)

Model Fit Quality:

Cases:
  MAE            : 16.3149
  RMSE           : 17.7635
  R2             : 0.0745
  MAPE           : 73.3007

Deaths:
  MAE            : 4.1250
  RMSE           : 5.7118
  R2             : -1.0901
  MAPE           : 75.0000
Observed vs Predicted:


,Date,Observed Cases,Predicted Cases,Observed Deaths,Predicted Deaths
0,2002-01-18,18,10.0,0,0.0
1,2002-10-04,12,35.4,0,0.0
2,2002-10-11,15,35.4,2,0.0
3,2002-10-17,18,35.4,2,0.0
4,2002-10-24,41,35.4,4,0.0
5,2002-10-31,45,35.5,4,0.0
6,2002-11-20,57,35.5,10,0.0
7,2002-11-28,60,35.5,11,0.0


## Visualize Model Fit